**Stocastic Gradient Descent (Batch Size 1)**

In [17]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

# Preparing for Data
print('==> Preparing data..')

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        ############################
        #### Put your code here ####
        ############################
        self.convnet = nn.Sequential(
            nn.Conv2d(1,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2),
            nn.Conv2d(16,120,5),
            nn.ReLU(),
            nn.Flatten())

        self.fc = nn.Sequential(
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10))
            # nn.LogSoftmax(dim=-1))

        
        
        ###########################
        #### End of your codes ####
        ###########################

    def forward(self, x):
        ############################
        #### Put your code here ####
        ############################
        y = self.convnet(x)
        out = self.fc(y)
        
        
        ###########################
        #### End of your codes ####
        ###########################
    

        return out



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        ############################
        #### Put your code here ####
        ############################
        optimizer.zero_grad()
        outputs = model(data)
        criterion = nn.CrossEntropyLoss()
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        
        ###########################
        #### End of your codes ####
        ###########################
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    time0 = time.time()
    # Training settings
    batch_size = 1
    epochs = 5
    lr = 0.01
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    
#This code to load the training and testing data came from https://blog.paperspace.com/writing-lenet5-from-scratch-in-python/ 

    train_dataset = torchvision.datasets.MNIST(root = './data',
                                           train = True,
                                           transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download = True)


    test_dataset = torchvision.datasets.MNIST(root = './data',
                                          train = False,
                                          transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                          download=True)


    train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


    test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)
    
    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0, weight_decay=0)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()


Streaming output truncated to the last 5000 lines.
Train Epoch: 5 [10040/60000 (17%)]	Loss: 0.000000
Train Epoch: 5 [10050/60000 (17%)]	Loss: 0.000016
Train Epoch: 5 [10060/60000 (17%)]	Loss: 0.000000
Train Epoch: 5 [10070/60000 (17%)]	Loss: 0.000000
Train Epoch: 5 [10080/60000 (17%)]	Loss: 0.000000
Train Epoch: 5 [10090/60000 (17%)]	Loss: 0.000000
Train Epoch: 5 [10100/60000 (17%)]	Loss: 0.000000
Train Epoch: 5 [10110/60000 (17%)]	Loss: 0.000000
Train Epoch: 5 [10120/60000 (17%)]	Loss: 0.000000
Train Epoch: 5 [10130/60000 (17%)]	Loss: 0.000009
Train Epoch: 5 [10140/60000 (17%)]	Loss: 0.000000
Train Epoch: 5 [10150/60000 (17%)]	Loss: 0.000000
Train Epoch: 5 [10160/60000 (17%)]	Loss: 0.000540
Train Epoch: 5 [10170/60000 (17%)]	Loss: 0.000002
Train Epoch: 5 [10180/60000 (17%)]	Loss: 0.000000
Train Epoch: 5 [10190/60000 (17%)]	Loss: 0.002139
Train Epoch: 5 [10200/60000 (17%)]	Loss: 0.000000
Train Epoch: 5 [10210/60000 (17%)]	Loss: 0.000000
Train Epoch: 5 [10220/60000 (17%)]	Loss: 0.000020

**Mini Batch Gradient Descent (Batch Size 128)**

In [16]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

# Preparing for Data
print('==> Preparing data..')

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        ############################
        #### Put your code here ####
        ############################
        self.convnet = nn.Sequential(
            nn.Conv2d(1,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2),
            nn.Conv2d(16,120,5),
            nn.ReLU(),
            nn.Flatten())

        self.fc = nn.Sequential(
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10))
            # nn.LogSoftmax(dim=-1))

        
        
        ###########################
        #### End of your codes ####
        ###########################

    def forward(self, x):
        ############################
        #### Put your code here ####
        ############################
        y = self.convnet(x)
        out = self.fc(y)
        
        
        ###########################
        #### End of your codes ####
        ###########################
    

        return out



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        ############################
        #### Put your code here ####
        ############################
        optimizer.zero_grad()
        outputs = model(data)
        criterion = nn.CrossEntropyLoss()
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        
        ###########################
        #### End of your codes ####
        ###########################
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    time0 = time.time()
    # Training settings
    batch_size = 128
    epochs = 5
    lr = 0.01
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    
#This code to load the training and testing data came from https://blog.paperspace.com/writing-lenet5-from-scratch-in-python/ 

    train_dataset = torchvision.datasets.MNIST(root = './data',
                                           train = True,
                                           transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download = True)


    test_dataset = torchvision.datasets.MNIST(root = './data',
                                          train = False,
                                          transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                          download=True)


    train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


    test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)
    
    model = LeNet().to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0, weight_decay=0)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()


==> Preparing data..
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.294770
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.308699
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.304215
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.294154
Train Epoch: 1 [5120/60000 (9%)]	Loss: 2.290165
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.287300
Train Epoch: 1 [7680/60000 (13%)]	Loss: 2.297962
Train Epoch: 1 [8960/60000 (15%)]	Loss: 2.292536
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.281795
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.286190
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.274302
Train Epoch: 1 [14080/60000 (23%)]	Loss: 2.284345
Train Epoch: 1 [15360/60000 (26%)]	Loss: 2.268158
Train Epoch: 1 [16640/60000 (28%)]	Loss: 2.272161
Train Epoch: 1 [17920/60000 (30%)]	Loss: 2.270352
Train Epoch: 1 [19200/60000 (32%)]	Loss: 2.260839
Train Epoch: 1 [20480/60000 (34%)]	Loss: 2.263937
Train Epoch: 1 [21760/60000 (36%)]	Loss: 2.251301
Train Epoch: 1 [23040/60000 (38%)]	Loss: 2.242730
Train Epoch: 1 [24320/60000 (41%)]	Loss: 2.23

**RMSProp**

In [19]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

# Preparing for Data
print('==> Preparing data..')

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        ############################
        #### Put your code here ####
        ############################
        self.convnet = nn.Sequential(
            nn.Conv2d(1,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2),
            nn.Conv2d(16,120,5),
            nn.ReLU(),
            nn.Flatten())

        self.fc = nn.Sequential(
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10))
            # nn.LogSoftmax(dim=-1))

        
        
        ###########################
        #### End of your codes ####
        ###########################

    def forward(self, x):
        ############################
        #### Put your code here ####
        ############################
        y = self.convnet(x)
        out = self.fc(y)
        
        
        ###########################
        #### End of your codes ####
        ###########################
    

        return out



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        ############################
        #### Put your code here ####
        ############################
        optimizer.zero_grad()
        outputs = model(data)
        criterion = nn.CrossEntropyLoss()
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        
        ###########################
        #### End of your codes ####
        ###########################
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    time0 = time.time()
    # Training settings
    batch_size = 128
    epochs = 5
    lr = 0.01
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    
#This code to load the training and testing data came from https://blog.paperspace.com/writing-lenet5-from-scratch-in-python/ 

    train_dataset = torchvision.datasets.MNIST(root = './data',
                                           train = True,
                                           transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download = True)


    test_dataset = torchvision.datasets.MNIST(root = './data',
                                          train = False,
                                          transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                          download=True)


    train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


    test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)
    
    model = LeNet().to(device)
    optimizer = torch.optim.RMSprop(model.parameters(), lr=lr, weight_decay=0, momentum=0, centered=False)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()


==> Preparing data..
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.294770
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.264327
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.190665
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.376672
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.061951
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.867166
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.593477
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.831129
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.303473
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.372170
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.222695
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.328274
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.211997
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.364190
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.308643
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.224505
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.122232
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.146345
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.111920
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.22

**Adagrad**

In [20]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time

# Preparing for Data
print('==> Preparing data..')

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        ############################
        #### Put your code here ####
        ############################
        self.convnet = nn.Sequential(
            nn.Conv2d(1,6,5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(6,16,5),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2),
            nn.Conv2d(16,120,5),
            nn.ReLU(),
            nn.Flatten())

        self.fc = nn.Sequential(
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10))
            # nn.LogSoftmax(dim=-1))

        
        
        ###########################
        #### End of your codes ####
        ###########################

    def forward(self, x):
        ############################
        #### Put your code here ####
        ############################
        y = self.convnet(x)
        out = self.fc(y)
        
        
        ###########################
        #### End of your codes ####
        ###########################
    

        return out



def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        ############################
        #### Put your code here ####
        ############################
        optimizer.zero_grad()
        outputs = model(data)
        criterion = nn.CrossEntropyLoss()
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
        
        
        ###########################
        #### End of your codes ####
        ###########################
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

def main():
    time0 = time.time()
    # Training settings
    batch_size = 128
    epochs = 5
    lr = 0.01
    no_cuda = True
    save_model = False
    use_cuda = not no_cuda and torch.cuda.is_available()
    torch.manual_seed(100)
    device = torch.device("cuda" if use_cuda else "cpu")
    
    
#This code to load the training and testing data came from https://blog.paperspace.com/writing-lenet5-from-scratch-in-python/ 

    train_dataset = torchvision.datasets.MNIST(root = './data',
                                           train = True,
                                           transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download = True)


    test_dataset = torchvision.datasets.MNIST(root = './data',
                                          train = False,
                                          transform = transforms.Compose([
                                                  transforms.Resize((32,32)),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize(mean = (0.1325,), std = (0.3105,))]),
                                          download=True)


    train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


    test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)
    
    model = LeNet().to(device)
    optimizer = torch.optim.Adagrad(model.parameters(), lr=lr, lr_decay=0, weight_decay=0, initial_accumulator_value=0)

    for epoch in range(1, epochs + 1):
        train( model, device, train_loader, optimizer, epoch)
        test( model, device, test_loader)

    if (save_model):
        torch.save(model.state_dict(),"cifar_lenet.pt")
    time1 = time.time() 
    print ('Traning and Testing total excution time is: %s seconds ' % (time1-time0))   
if __name__ == '__main__':
    main()


==> Preparing data..
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.294770
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.530969
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.562097
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.291613
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.307380
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.225736
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.179722
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.394915
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.123502
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.105304
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.119261
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.212007
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.070707
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.269905
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.176177
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.105208
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.087286
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.062994
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.063209
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.18